In [ ]:
import cv2 
import csv
import glob
import natsort
import numpy as np
import re
import pickle

In [ ]:
user = 'gowtham'

In [ ]:
def get_numbers_from_filename(filename):
    'utility function to get first number from a file name'
    return re.search(r'\d+', filename).group(0)

In [ ]:
def markSafeDurationForVideo(video_path):

    cv2.namedWindow(video_path, cv2.WINDOW_AUTOSIZE)
    font = cv2.FONT_HERSHEY_COMPLEX 
    fontScale = 1  
    fontColor = (255,255,255) 
    lineType = 2 
    scanned_video = cv2.VideoCapture(video_path)
    no_frames = int(scanned_video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_count = -1
    flag = 1

    ss = []  
    se = [] 
    safe_duration_list = [] 

    if (scanned_video.isOpened()== False): 
        raise Exception("check the videopath or error opening video")

    while(scanned_video.isOpened()):

        ret, frame = scanned_video.read() 
        key = cv2.waitKey(34)
        
        if(ret == True):

            if(frame_count == -1):
                while True:

                    key_init = cv2.waitKey(33)
                    cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
                    cv2.imshow(video_path, frame)

                    if key_init == ord('a'):  
                        break
            
            frame_count = frame_count + 1
            
            cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
            cv2.imshow(video_path,frame)

            if key == ord('p'): 

                while True:
                    
                    key2 = cv2.waitKey(33)

                    cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
                    cv2.imshow(video_path, frame)
                    
                    if key2 == ord('p'): 
                        break
            
            if key == ord('r'): 
                ss.append(0)
                if (len(ss) != 1):
                    print("You have made some mistake")
                    flag = 0
                    break
                    
                safe_duration_list.append(0)
            
            if key == ord('s'):

                ss.append(frame_count)
                if (len(ss) != len(se) + 1):
                    print("You have made some mistake")
                    flag = 0
                    break
                    
                safe_duration_list.append(frame_count)
                
            if key == ord('e'):

                se.append(frame_count)
                if (len(se) != len(ss)):
                    print("You have made some mistake")
                    flag = 0
                    break
                safe_duration_list.append(frame_count)
            
            if key == ord('q'): 
                break

        else:
            break


    scanned_video.release()
    cv2.destroyAllWindows()

    if(len(safe_duration_list) % 2 !=0):
        safe_duration_list.append(no_frames - 1)
    
    return safe_duration_list, flag


In [ ]:

def save_labels_csv(videos_list, csv_file, pickle_list_path):    
    csv_writer = csv.writer(open(csv_file,"w"))
    open_file = open(pickle_list_path, "rb")
    pickle_list = pickle.load(open_file)
    open_file.close()
    
    for video_path in videos_list:
        
        print('\nstarting for', video_path)
        video_no = int(get_numbers_from_filename(video_path)) 
        ready_flag = str.lower(input('type "y" if you are ready to annotate the video and "n" to quit the program\n'))
        if(ready_flag == 'y'):
            correct_flag = 1
            surity_flag = 0
            while(surity_flag == 0 or correct_flag == 0):
                safe_duration_list, correct_flag = markSafeDurationForVideo(video_path)
                if(correct_flag == 0):
                    print("press keys carefully this time!")
                    input("press any key when you are ready")
                else:
                    surity_flag = int(input('Enter 0 to repeat the annotations process and 1 to proceed to next video\n'))
                if(surity_flag == 1 and correct_flag == 1):
                    break

            pickle_list[video_no-1] = safe_duration_list

            print("safe_duration_list", safe_duration_list)            
        else:
            
            print('\nTake a break! You must be tired')
            break
            
    open_file = open(pickle_list_path, "wb")
    pickle.dump(pickle_list, open_file)
    open_file.close()
    csv_writer.writerows(pickle_list)
    print("list and csv file saved")

In [ ]:


if(user == 'gowtham'):
    video_folder = '/users/gowtham/downloads/majorproject/Data/videos'
    videos_list = glob.glob(video_folder + '/video*.MOV')
    videos_list = (natsort.natsorted(videos_list))
    pickle_list_path = 'C/users/gowtham/downloads/majorproject/Data/labels_framewise_list.pkl'
    csv_file_path = '/users/gowtham/downloads/majorproject/Data/Data/labels_framewise_csv.csv'

In [ ]:
save_labels_csv(videos_list, csv_file_path, pickle_list_path)

In [ ]:
open_file = open(pickle_list_path, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

print(len(loaded_list),'\n', loaded_list)
print(loaded_list[np.array([0,1,5,7,9])])

In [ ]:
def visualize_annotations(video, safe_duration_list):


    cap = cv2.VideoCapture(video)
    cv2.namedWindow(video, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(video, height=650, width=1156)

    no_safe_durations = int(len(safe_duration_list)/2)
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    labels = [0]*no_frames
    for i in range(no_safe_durations):
        ss = safe_duration_list[i*2]
        se = safe_duration_list[i*2 + 1]
        labels[ss:se+1] = [1]*(se+1-ss)
    
    frame_count = -1
    


    while(cap.isOpened()):

        success,frame = cap.read()
        #print(i)

        if success == True:

            if(frame_count == -1):
                while True:

                    key_init = cv2.waitKey(25)
                    cv2.imshow(video, frame)

                    if key_init == ord('a'):
                        break

            frame_count = frame_count + 1
            

            if(labels[frame_count] == 1):

                cv2.rectangle(frame, (1576,3), (1890,95), (0,200,0), thickness=-1)
                frame_save = cv2.putText(frame, 'SAFE', (1580,90), cv2.FONT_HERSHEY_SIMPLEX, 4, (255,255,255), 6, cv2.LINE_AA)
                cv2.imshow(video, frame_save)
                cv2.waitKey(34)

            else:

                cv2.rectangle(frame, (1396,3), (1890,95), (0,0,200), thickness=-1)
                frame_save = cv2.putText(frame, 'UNSAFE', (1400,90), cv2.FONT_HERSHEY_SIMPLEX, 4, (255,255,255), 6, cv2.LINE_AA)
                cv2.imshow(video, frame_save)
                cv2.waitKey(34)

        else:
            break

    
    cap.release()

    cv2.destroyAllWindows()


In [ ]:
def verify_annotations(pickle_list, videos_list):

    open_file = open(pickle_list, "rb")
    labels_framewise = (pickle.load(open_file))[10:]
    open_file.close()

    for i in range(len(videos_list)):

        safe_duration_list = labels_framewise[i]
        video = videos_list[i]

        print('\nstarting for', video)
        ready_flag = str.lower(input('type y if you are ready to start the video and n to quit the program\n'))


        if(ready_flag == 'y'):

            surity_flag = 0
            while surity_flag == 0:
                visualize_annotations(video, safe_duration_list)
                surity_flag = int(input('type 1 if you are sure and 0 to replay the video\n'))

        else:

            print('\nTake a break! You must be tired')
            break

In [ ]:
verify_annotations(pickle_list_path, videos_list[10:])